# Vertex AI: Qwik Start

## Objetivos de aprendizado

* Treine um modelo do TensorFlow localmente em um [**Vertex Notebook**](https://cloud.google.com/vertex-ai/docs/general/notebooks?hl=sv) hospedado.
* Crie um artefato [**conjunto de dados tabular gerenciado**](https://cloud.google.com/vertex-ai/docs/training/using-managed-datasets?hl=sv) para rastreamento de experimentos.
* Conteinerize seu código de treinamento com [**Cloud Build**](https://cloud.google.com/build) e envie-o para [**Google Cloud Artifact Registry**](https://cloud.google.com/artifact-registry).
* Execute um [**trabalho de treinamento personalizado da Vertex AI**](https://cloud.google.com/vertex-ai/docs/training/custom-training) com seu contêiner de modelo personalizado.
* Use [**Vertex TensorBoard**](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) para visualizar o desempenho do modelo.
* Implante seu modelo treinado em um [**Vertex Online Prediction Endpoint**](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions) para fornecer previsões.
* Solicite uma previsão e explicação online e veja a resposta.

## Introdução: previsão do valor da vida útil do cliente (CLV) com BigQuery e TensorFlow na Vertex AI

Neste laboratório, você usará o [BigQuery](https://cloud.google.com/bigquery) para processamento e análise exploratória de dados e o [Vertex AI](https://cloud.google.com/vertex-ai ) para treinar e implantar um modelo TensorFlow Regressor personalizado para prever o valor da vida útil do cliente (CLV). O objetivo do laboratório é apresentar a Vertex AI por meio de um caso de uso real de alto valor - CLV preditivo. Você começará com um fluxo de trabalho local do BigQuery e TensorFlow com o qual já deve estar familiarizado e progredirá para treinar e implantar seu modelo na nuvem com a Vertex AI.

![Vertex AI](./images/vertex-ai-overview.png "Vertex AI Overview")

A Vertex AI é a plataforma unificada de próxima geração do Google Cloud para desenvolvimento de machine learning e a sucessora da AI Platform anunciada no Google I/O em maio de 2021. Ao desenvolver soluções de machine learning na Vertex AI, você pode aproveitar os mais recentes componentes pré-criados de ML e AutoML para aumentar significativamente a produtividade do desenvolvimento, a capacidade de dimensionar seu fluxo de trabalho e a tomada de decisões com seus dados e acelerar o tempo de retorno.

### CLV preditivo: quanto valor monetário os clientes existentes trarão para a empresa no futuro

O CLV preditivo é um caso de uso comercial de ML de alto impacto. CLV é o valor passado de um cliente mais seu valor futuro previsto. O objetivo do CLV preditivo é prever quanto valor monetário um usuário trará para o negócio em um intervalo de tempo futuro definido com base em transações históricas.

Ao conhecer o CLV, você pode desenvolver estratégias de ROI positivas e tomar decisões sobre quanto dinheiro investir na aquisição de novos clientes e na retenção dos existentes para aumentar a receita e o lucro.

Depois que seu modelo de ML for um sucesso, você poderá usar os resultados para identificar os clientes com maior probabilidade de gastar dinheiro do que os outros e fazê-los responder às suas ofertas e descontos com maior frequência. Esses clientes, com maior valor vitalício, são seu principal alvo de marketing para aumentar a receita.

Ao usar a abordagem de aprendizado de máquina para prever o valor de seus clientes que você usará neste laboratório, você pode priorizar suas próximas ações, como as seguintes:

* Decida quais clientes segmentar com publicidade para aumentar a receita.
* Identifique quais segmentos de clientes são mais lucrativos e planeje como mover os clientes de um segmento para outro.

Sua tarefa é prever o valor futuro dos clientes existentes com base em seu histórico de transações conhecido.

![CLV](./images/clv-rfm.svg "Valor vitalício do cliente")
Fonte: [Centro de Arquitetura em Nuvem - Prevendo o valor da vida útil do cliente com AI Platform: treinando os modelos](https://cloud.google.com/architecture/clv-prediction-with-offline-training-train)

Há uma forte correlação positiva entre a recência, a frequência e a quantia gasta em cada compra que cada cliente faz e seu CLV. Consequentemente, você aproveitará esses recursos em seu modelo de ML. Para este laboratório, eles são definidos como:

* **Recência**: o tempo entre a última compra e hoje, representado pela distância entre o círculo mais à direita e a linha pontilhada vertical marcada como "Agora".
* **Frequência**: O tempo entre as compras, representado pela distância entre os círculos em uma única linha.
* **Monetary**: a quantia de dinheiro gasta em cada compra, representada pelo tamanho do círculo. Esse valor pode ser o valor médio do pedido ou a quantidade de produtos que o cliente solicitou.

## Configurações

### Definição de constantes

In [ ]:
# Adicione as dependências da biblioteca instalada à variável Python PATH.
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

In [ ]:
# Recupere e defina as variáveis de ambiente PROJECT_ID e REGION.
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
REGION = 'us-central1'

In [ ]:
# Crie um bucket globalmente exclusivo do Google Cloud Storage para armazenamento de artefatos.
GCS_BUCKET = f"{PROJECT_ID}-bucket"

In [ ]:
!gsutil mb -l $REGION gs://$GCS_BUCKET

### Importar bibliotecas

In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from google.cloud import aiplatform

### Inicialize o cliente Vertex Python SDK

Importe o Vertex SDK for Python para seu ambiente Python e inicialize-o.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=f"gs://{GCS_BUCKET}")

## Baixe e processe os dados do laboratório no BigQuery

### Conjunto de dados

Neste laboratório, você usará o [conjunto de dados de varejo on-line] disponível publicamente (https://archive.ics.uci.edu/ml/datasets/online+retail) do UCI Machine Learning Repository. Este conjunto de dados contém 541.909 transações transnacionais de clientes ocorrendo entre (AAAA-MM-DD) 2010-12-01 e 2011-12-09 para um varejista sem loja registrado e baseado no Reino Unido. A empresa vende principalmente presentes exclusivos para todas as ocasiões. Muitos dos clientes da empresa são atacadistas.

**Citação**
Dua, D. e Karra Taniskidou, E. (2017). UCI Machine Learning Repository http://archive.ics.uci.edu/ml. Irvine, CA: Universidade da Califórnia, Escola de Informação e Ciência da Computação.

Este laboratório também é inspirado na série de guias do arquiteto do Google Cloud [Prevendo o valor da vida útil do cliente com AI Platform: introdução](https://cloud.google.com/architecture/clv-prediction-with-offline-training-intro).

### Ingestão de dados

Execute o comando abaixo para ingerir os dados do laboratório do repositório UCI Machine Learning no `Cloud Storage` e, em seguida, faça o upload para o `BigQuery` para processamento de dados. Os scripts de ingestão e processamento de dados estão disponíveis na pasta `utils` no diretório lab.

In [ ]:
# constantes do BigQuery. Por favor, deixe-os inalterados.
BQ_DATASET_NAME="online_retail"
BQ_RAW_TABLE_NAME="online_retail_clv_raw"
BQ_CLEAN_TABLE_NAME="online_retail_clv_clean"
BQ_ML_TABLE_NAME="online_retail_clv_ml"
BQ_URI=f"bq://{PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_ML_TABLE_NAME}"

**Observação**: este script Python levará cerca de 2 a 3 minutos para baixar e processar o arquivo de dados do laboratório. Siga junto com a saída de registro na célula abaixo.

In [ ]:
!python utils/data_download.py \
  --PROJECT_ID={PROJECT_ID} \
  --GCS_BUCKET={GCS_BUCKET} \
  --BQ_RAW_TABLE_NAME={BQ_RAW_TABLE_NAME} \
  --BQ_CLEAN_TABLE_NAME={BQ_CLEAN_TABLE_NAME} \
  --BQ_ML_TABLE_NAME={BQ_ML_TABLE_NAME} \
  --URL="https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online Retail.xlsx"

### Processamento de Dados

Como é o caso de muitos conjuntos de dados do mundo real, o conjunto de dados do laboratório exigiu alguma limpeza para você utilizar esses dados históricos de transações do cliente para CLV preditivo.

As seguintes alterações foram aplicadas:

* Mantenha apenas registros que tenham um ID de cliente.
* Transações agregadas por dia a partir de faturas.
* Mantenha apenas registros que tenham quantidades de pedidos e valores monetários positivos.
* Agregue transações por ID do cliente e calcule a atualidade, a frequência, os recursos monetários, bem como a meta de previsão.

**Características**:
- `customer_country` (CATEGÓRICO): país de compra do cliente.
- `n_purchases` (NUMERIC): quantidade de compras realizadas na janela de funcionalidades. (frequência)
- `avg_purchase_size` (NUMERIC): contagem média de compra de unidades na janela de recursos. (monetário)
- `avg_purchase_revenue` (NUMERIC): valor médio de compra em GBP na janela de recursos. (monetário)
- `customer_age` (NUMERIC): dias a partir da primeira compra na janela de recursos.
- `days_since_last_purchase` (NUMERIC): dias a partir da compra mais recente na janela de recursos. (recência)

**Alvo**:
- `target_monetary_value_3M` (NUMERIC): receita do cliente de toda a janela de estudo, incluindo recursos e janelas de previsão.

Observação: este laboratório demonstra uma maneira simples de usar um DNN para prever o valor monetário CLV do cliente três meses à frente com base apenas no histórico de transações históricas do conjunto de dados disponível. Fatores adicionais a serem considerados na prática ao usar o CLV para informar as intervenções incluem custos de aquisição do cliente, margens de lucro e taxas de desconto para chegar ao valor presente dos fluxos de caixa futuros do cliente. Um dos benefícios de uma DNN em relação às abordagens tradicionais de modelagem probabilística é sua capacidade de incorporar recursos categóricos e não estruturados adicionais; esta é uma grande oportunidade de engenharia de recursos para explorar além deste laboratório, que apenas explora os recursos numéricos RFM.

## Análise exploratória de dados (EDA) no BigQuery

Abaixo, você usará o BigQuery deste notebook para fazer análises exploratórias de dados para conhecer esse conjunto de dados e identificar oportunidades para limpeza de dados e engenharia de recursos.

### Recência: há quanto tempo os clientes compraram?

In [ ]:
%%bigquery recency

SELECT 
  days_since_last_purchase
FROM 
  `online_retail.online_retail_clv_ml`

In [ ]:
recency.describe()

In [ ]:
recency.hist(bins=100);

No gráfico, há claramente alguns grupos de clientes diferentes aqui, como clientes fiéis que fizeram compras nos últimos dias, bem como clientes inativos que não compram há mais de 250 dias. Usando previsões e insights de CLV, você pode criar estratégias de marketing e intervenções promocionais para melhorar a recência da compra do cliente e reativar clientes inativos.

### Frequência: com que frequência os clientes compram?

In [ ]:
%%bigquery frequency

SELECT
  n_purchases
FROM
  `online_retail.online_retail_clv_ml`

In [ ]:
frequency.describe()

In [ ]:
frequency.hist(bins=100);

No gráfico e nos quantis, você pode ver que metade dos clientes tem menos ou igual a apenas 2 compras. Você também pode dizer pelas compras médias > compras medianas e compras máximas de 81 que existem clientes, provavelmente atacadistas, que fizeram significativamente mais compras. Isso deve fazer com que você já esteja pensando em oportunidades de engenharia de recursos, como segmentação de compras e remoção ou recorte de clientes atípicos. Você também pode explorar estratégias de modelagem alternativas para CLV em novos clientes que fizeram apenas uma compra, pois a abordagem demonstrada neste laboratório terá um desempenho melhor em clientes com mais histórico de transações de relacionamento.

### Monetário: quanto os clientes estão gastando?

In [ ]:
%%bigquery monetary

SELECT
  target_monetary_value_3M
FROM
`online_retail.online_retail_clv_ml`

In [ ]:
monetary.describe()

In [ ]:
monetary['target_monetary_value_3M'].plot(kind='box', title="Target Monetary Value 3M: wide range, long right tail distribution", grid=True);

No gráfico e nas estatísticas resumidas, você pode ver que há uma ampla variação no valor monetário do cliente, variando de 2,90 a 268.478 libras esterlinas. Olhando para os quantis, fica claro que existem alguns clientes atípicos cujo valor monetário é maior que 3 desvios padrão da média. Com esse pequeno conjunto de dados, é recomendável remover esses valores discrepantes do cliente para tratá-los separadamente, alterar a função de perda do seu modelo para ser mais resistente a discrepâncias, registrar o recurso de destino ou recortar seus valores para um limite máximo. Você também deve revisar seus requisitos de negócios CLV para ver se o valor monetário do cliente e reenquadrar isso como um problema de classificação de ML atenderia às suas necessidades.

### Estabeleça uma linha de base de desempenho de modelo simples

Para avaliar o desempenho de seu modelo TensorFlow DNN Regressor personalizado que você criará nas próximas etapas, é uma prática recomendada de ML estabelecer uma linha de base de desempenho simples. Abaixo está uma linha de base SQL simples que multiplica o gasto médio de compra de um cliente composto por sua taxa de compra diária e calcula métricas de regressão padrão.

In [ ]:
%%bigquery

WITH
  day_intervals AS (
  SELECT
      customer_id,
      DATE_DIFF(DATE('2011-12-01'), DATE('2011-09-01'), DAY) AS target_days,
      DATE_DIFF(DATE('2011-09-01'), MIN(order_date), DAY) AS feature_days,
  FROM
    `online_retail.online_retail_clv_clean`
  GROUP BY
      customer_id
  ),
    
  predicted_clv AS (
  SELECT
      customer_id,
      AVG(avg_purchase_revenue) * (COUNT(n_purchases) * (1 + SAFE_DIVIDE(COUNT(target_days),COUNT(feature_days)))) AS predicted_monetary_value_3M,
      SUM(target_monetary_value_3M) AS target_monetary_value_3M
  FROM
    `online_retail.online_retail_clv_ml`
  LEFT JOIN day_intervals USING(customer_id)
  GROUP BY
      customer_id
  )

# Calcule métricas gerais de regressão linear
SELECT
  ROUND(AVG(ABS(predicted_monetary_value_3M - target_monetary_value_3M)), 2) AS MAE,
  ROUND(AVG(POW(predicted_monetary_value_3M - target_monetary_value_3M, 2)), 2) AS MSE,
  ROUND(SQRT(AVG(POW(predicted_monetary_value_3M - target_monetary_value_3M, 2))), 2) AS RMSE
FROM
  predicted_clv

Esses resultados da linha de base fornecem suporte adicional para o forte impacto dos valores discrepantes. O MSE extremamente alto vem da penalidade exponencial aplicada a previsões perdidas e da magnitude do erro em algumas previsões.

Em seguida, você deve traçar os resultados da linha de base para ter uma noção das áreas de oportunidade para seu modelo de ML.

In [ ]:
%%bigquery baseline

WITH
  day_intervals AS (
  SELECT
      customer_id,
      DATE_DIFF(DATE('2011-12-01'), DATE('2011-09-01'), DAY) AS target_days,
      DATE_DIFF(DATE('2011-09-01'), MIN(order_date), DAY) AS feature_days,
  FROM
    `online_retail.online_retail_clv_clean`
  GROUP BY
      customer_id
  ),
    
  predicted_clv AS (
  SELECT
      customer_id,
      AVG(avg_purchase_revenue) * (COUNT(n_purchases) * (1 + SAFE_DIVIDE(COUNT(target_days),COUNT(feature_days)))) AS predicted_monetary_value_3M,
      SUM(target_monetary_value_3M) AS target_monetary_value_3M
  FROM
    `online_retail.online_retail_clv_ml`
  INNER JOIN day_intervals USING(customer_id)
  GROUP BY
      customer_id
  )

SELECT
 *
FROM
  predicted_clv

In [ ]:
baseline.head()

In [ ]:
ax = baseline.plot(kind='scatter',
                   x='predicted_monetary_value_3M', 
                   y='target_monetary_value_3M',
                   title='Actual vs. Predicted customer 3-month monetary value',
                   figsize=(5,5),
                   grid=True)

lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

# now plot both limits against eachother
ax.plot(lims, lims, 'k-', alpha=0.5, zorder=0)
ax.set_aspect('equal')
ax.set_xlim(lims)
ax.set_ylim(lims);

## Treine um modelo do TensorFlow localmente

Agora que você tem uma linha de base simples para comparar seu desempenho, treine um TensorFlow Regressor para prever o CLV.

In [ ]:
%%bigquery

SELECT data_split, COUNT(*)
FROM `online_retail.online_retail_clv_ml`
GROUP BY data_split

In [ ]:
%%bigquery clv

SELECT *
FROM `online_retail.online_retail_clv_ml`

In [ ]:
clv_train = clv.loc[clv.data_split == 'TRAIN', :]
clv_dev = clv.loc[clv.data_split == 'VALIDATE', :]
clv_test = clv.loc[clv.data_split == 'TEST', :]

In [ ]:
# Constantes de treinamento do modelo.
# Padrão de design de épocas virtuais:
# https://medium.com/google-cloud/ml-design-pattern-3-virtual-epochs-f842296de730
N_TRAIN_EXAMPLES = 2638
STOP_POINT = 20.0
TOTAL_TRAIN_EXAMPLES = int(STOP_POINT * N_TRAIN_EXAMPLES)
BATCH_SIZE = 32
N_CHECKPOINTS = 10
STEPS_PER_EPOCH = (TOTAL_TRAIN_EXAMPLES // (BATCH_SIZE*N_CHECKPOINTS))

NUMERIC_FEATURES = [
    "n_purchases",
    "avg_purchase_size",
    "avg_purchase_revenue",
    "customer_age",
    "days_since_last_purchase",
]

LABEL = "target_monetary_value_3M"

In [ ]:
def df_dataset(df):
    """Transform Pandas Dataframe to TensorFlow Dataset."""
    return tf.data.Dataset.from_tensor_slices((df[NUMERIC_FEATURES].to_dict('list'), df[LABEL].values))

In [ ]:
trainds = df_dataset(clv_train).prefetch(1).batch(BATCH_SIZE).repeat()
devds = df_dataset(clv_dev).prefetch(1).batch(BATCH_SIZE)
testds = df_dataset(clv_test).prefetch(1).batch(BATCH_SIZE)

In [ ]:
def rmse(y_true, y_pred):
    """Custom RMSE regression metric."""
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))


def build_model():
    """Build and compile a TensorFlow Keras Regressor."""
    # Defina tensores de recursos de entrada e camadas de entrada.
    feature_columns = [
        tf.feature_column.numeric_column(key=feature)
        for feature in NUMERIC_FEATURES
    ]
    
    input_layers = {
        feature.key: tf.keras.layers.Input(name=feature.key, shape=(), dtype=tf.float32)
        for feature in feature_columns
    }
     
    # Keras Functional API: https://keras.io/guides/functional_api
    inputs = tf.keras.layers.DenseFeatures(feature_columns, name='inputs')(input_layers)
    d1 = tf.keras.layers.Dense(256, activation=tf.nn.relu, name='d1')(inputs)
    d2 = tf.keras.layers.Dropout(0.2, name='d2')(d1)  
    # Nota: a saída de um único neurônio para regressão.
    output = tf.keras.layers.Dense(1, name='output')(d2)
    
    model = tf.keras.Model(input_layers, output, name='online-retail-clv')
    
    optimizer = tf.keras.optimizers.Adam(0.001)    
    
    # Note: A perda de MAE é mais resistente a valores discrepantes do que MSE.
    model.compile(loss=tf.keras.losses.MAE,
                  optimizer=optimizer,
                  metrics=[['mae', 'mse', rmse]])
    
    return model

model = build_model()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='./local-training/tensorboard',
    histogram_freq=1)

earlystopping_callback = tf.keras.callbacks.EarlyStopping(patience=1)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./local-training/checkpoints',
    save_weights_only=True,
    monitor='val_loss',
    mode='min')

In [ ]:
history = model.fit(trainds,
                    validation_data=devds,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    epochs=N_CHECKPOINTS,
                    callbacks=[[tensorboard_callback,
                                earlystopping_callback,
                                checkpoint_callback]])

In [ ]:
LOSS_COLS = ["loss", "val_loss"]

pd.DataFrame(history.history)[LOSS_COLS].plot();

In [ ]:
train_pred = model.predict(df_dataset(clv_train).prefetch(1).batch(BATCH_SIZE))
dev_pred = model.predict(devds)
test_pred = model.predict(testds)

In [ ]:
train_results = pd.DataFrame({'actual': clv_train['target_monetary_value_3M'].to_numpy(), 'predicted': np.squeeze(train_pred)}, columns=['actual', 'predicted'])
dev_results = pd.DataFrame({'actual': clv_dev['target_monetary_value_3M'].to_numpy(), 'predicted': np.squeeze(dev_pred)}, columns=['actual', 'predicted'])
test_results = pd.DataFrame({'actual': clv_test['target_monetary_value_3M'].to_numpy(), 'predicted': np.squeeze(test_pred)}, columns=['actual', 'predicted'])

In [ ]:
#Gráficos de calibração de previsão de modelo.
fig, (train_ax, dev_ax, test_ax) = plt.subplots(1, 3, figsize=(15,15))

train_results.plot(kind='scatter',
                  x='predicted',
                  y='actual',
                  title='Train: act vs. pred customer 3M monetary value',
                  grid=True,
                  ax=train_ax)

train_lims = [
    np.min([train_ax.get_xlim(), train_ax.get_ylim()]),  # min de ambos os eixos
    np.max([train_ax.get_xlim(), train_ax.get_ylim()]),  # max de ambos os eixos
]

train_ax.plot(train_lims, train_lims, 'k-', alpha=0.5, zorder=0)
train_ax.set_aspect('equal')
train_ax.set_xlim(train_lims)
train_ax.set_ylim(train_lims)

dev_results.plot(kind='scatter',
                  x='predicted',
                  y='actual',
                  title='Dev: act vs. pred customer 3M monetary value',
                  grid=True,
                  ax=dev_ax)

dev_lims = [
    np.min([dev_ax.get_xlim(), dev_ax.get_ylim()]),  # min de ambos os eixos
    np.max([dev_ax.get_xlim(), dev_ax.get_ylim()]),  # max de ambos os eixos
]

dev_ax.plot(dev_lims, dev_lims, 'k-', alpha=0.5, zorder=0)
dev_ax.set_aspect('equal')
dev_ax.set_xlim(dev_lims)
dev_ax.set_ylim(dev_lims)

test_results.plot(kind='scatter',
                  x='predicted',
                  y='actual',
                  title='Test: act vs. pred customer 3M monetary value',
                  grid=True,
                  ax=test_ax)

test_lims = [
    np.min([test_ax.get_xlim(), test_ax.get_ylim()]),  # min de ambos os eixos
    np.max([test_ax.get_xlim(), test_ax.get_ylim()]),  # max de ambos os eixos
]

test_ax.plot(test_lims, test_lims, 'k-', alpha=0.5, zorder=0)
test_ax.set_aspect('equal')
test_ax.set_xlim(test_lims)
test_ax.set_ylim(test_lims);

Você treinou um modelo melhor do que sua linha de base. Conforme indicado nos gráficos acima, ainda há oportunidades adicionais de engenharia de recursos e limpeza de dados para melhorar o desempenho do seu modelo em clientes com CLV. Algumas opções incluem lidar com esses clientes como uma tarefa de previsão separada, aplicar uma transformação de log ao seu destino, recortar seu valor ou descartar esses clientes todos juntos para melhorar o desempenho do modelo.

Agora você trabalhará levando esse fluxo de trabalho local do TensorFlow para a nuvem com a Vertex AI.

## Crie um conjunto de dados tabular gerenciado de sua fonte de dados do BigQuery

[**Conjuntos de dados gerenciados da Vertex AI**](https://cloud.google.com/vertex-ai/docs/datasets/prepare-tabular) podem ser usados para treinar modelos AutoML ou modelos treinados personalizados.

Você criará um [**conjunto de dados de regressão tabular**](https://cloud.google.com/vertex-ai/docs/datasets/bp-tabular) para gerenciar o compartilhamento e os metadados do conjunto de dados deste laboratório armazenado no BigQuery. Conjuntos de dados gerenciados permitem que você crie um vínculo claro entre seus dados e modelos treinados personalizados e forneça estatísticas descritivas e divisão automática ou manual em conjuntos de treinamento, teste e validação.

Neste laboratório, a etapa de processamento de dados já criou uma coluna `data_split` manual em nossa tabela BQ ML usando [funções de hash do BigQuery](https://towardsdatascience.com/ml-design-pattern-5-repeatable-sampling-c0ccb2889f39) para amostragem repetível.

In [ ]:
tabular_dataset = aiplatform.TabularDataset.create(display_name="online-retail-clv", bq_source=f"{BQ_URI}")

## Fluxo de trabalho de treinamento de modelo de ML personalizado da Vertex AI

Há duas maneiras de treinar um modelo personalizado na Vertex AI:

Antes de enviar um trabalho de treinamento personalizado, um trabalho de ajuste de hiperparâmetros ou um pipeline de treinamento para a Vertex AI, você precisa criar um aplicativo de treinamento em Python ou um contêiner personalizado para definir o código de treinamento e as dependências que deseja executar na Vertex AI.

**1. Use um contêiner predefinido do Google Cloud**: se você usar um contêiner predefinido do Vertex AI, escreverá um script Python `task.py` ou um pacote Python para instalar na imagem do contêiner que define seu código para treinar um modelo personalizado. Consulte [Creating a Python training application for a pre-built container](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container) para obter mais detalhes sobre como estruturar seu código Python. Escolha esta opção se um contêiner pré-criado já contiver as bibliotecas de treinamento de modelo de que você precisa, como `tensorflow` ou `xgboost`, e você estiver apenas fazendo treinamento e previsão de ML rapidamente. Você também pode especificar dependências adicionais do Python para instalar por meio do argumento `CustomTrainingJob(requirements=...`).

**2. Use sua própria imagem de contêiner personalizada**: se quiser usar seu próprio contêiner personalizado, você escreverá seus scripts de treinamento Python e um Dockerfile que contém instruções sobre seu código de modelo de ML, dependências e instruções de execução. Você criará seu contêiner personalizado com o Cloud Build, cujas instruções são especificadas em `cloudbuild.yaml` e publicará seu contêiner no Artifact Registry. Escolha esta opção se quiser empacotar seu código de modelo de ML com dependências em um contêiner para criar a execução como parte de um [Vertex Pipelines] portátil e escalonável (https://cloud.google.com/vertex-ai/docs/pipelines/introdução) fluxo de trabalho.

### Conteinerize seu código de treinamento de modelo

Nas próximas 5 etapas, você prosseguirá com **2. Use sua própria imagem de contêiner personalizada**.

Você criará seu contêiner de modelo personalizado sobre um [contêiner de aprendizagem profunda do Google Cloud](https://cloud.google.com/vertex-ai/docs/general/deep-learning) que contém versões testadas e otimizadas do código do modelo dependências como `tensorflow` e o SDK `google-cloud-bigquery`. Isso também oferece flexibilidade e permite gerenciar e compartilhar sua imagem de contêiner de modelo com outras pessoas para reutilização e reprodutibilidade em ambientes, além de permitir que você incorpore pacotes adicionais para seu aplicativo de ML. Por fim, ao empacotar seu código de modelo de ML junto com as dependências, você também tem um caminho de integração MLOps para o Vertex Pipelines.

Você passará pela criação da seguinte estrutura de projeto para seu código de modo ML:

```
|--/online-retail-clv-3M
   |--/trainer
      |--__init__.py
      |--model.py
      |--task.py
   |--Dockerfile
   |--cloudbuild.yaml
   |--requirements.txt
```

#### 1. Escreva um script de treinamento `model.py`

Primeiro, você organizará o código de treinamento do modelo TensorFlow local acima em um script de treinamento.

A maior mudança é que você utilizará a biblioteca [TensorFlow IO](https://www.tensorflow.org/io/tutorials/bigquery) para ler com desempenho do BigQuery diretamente no gráfico do modelo do TensorFlow durante o treinamento. Isso melhorará seu desempenho de treinamento em vez de executar a etapa intermediária de leitura do BigQuery em um Pandas Dataframe feito para conveniência acima.

In [ ]:
# este é o nome do subdiretório do modelo no qual você escreverá o código do modelo. Ele já está criado no diretório do seu laboratório.
MODEL_NAME="online-retail-clv-3M"

In [ ]:
%%writefile {MODEL_NAME}/trainer/model.py
import os
import logging
import tempfile
import tensorflow as tf
from explainable_ai_sdk.metadata.tf.v2 import SavedModelMetadataBuilder
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession


# Constantes de recursos do modelo.
NUMERIC_FEATURES = [
    "n_purchases",
    "avg_purchase_size",
    "avg_purchase_revenue",
    "customer_age",
    "days_since_last_purchase",
]

CATEGORICAL_FEATURES = [
    "customer_country"
]

LABEL = "target_monetary_value_3M"


def caip_uri_to_fields(uri):
    """Helper function to parse BQ URI."""
    # Remova o prefixo bq://.
    uri = uri[5:]
    project, dataset, table = uri.split('.')
    return project, dataset, table


def features_and_labels(row_data):
    """Helper feature and label mapping function for tf.data."""
    label = row_data.pop(LABEL)
    features = row_data
    return features, label


def read_bigquery(project, dataset, table):
    """TensorFlow IO BigQuery Reader."""
    tensorflow_io_bigquery_client = BigQueryClient()
    read_session = tensorflow_io_bigquery_client.read_session(
      parent="projects/" + project,
      project_id=project, 
      dataset_id=dataset,
      table_id=table,
      # Passe a lista de recursos e rótulos a serem selecionados no BQ.
      selected_fields=NUMERIC_FEATURES + [LABEL],
     # Forneça tipos de dados TensorFlow de saída para recursos e rótulos.
      output_types=[dtypes.int64, dtypes.float64, dtypes.float64, dtypes.int64, dtypes.int64] + [dtypes.float64],
      requested_streams=2)
    dataset = read_session.parallel_read_rows()
    transformed_ds = dataset.map(features_and_labels)
    return transformed_ds


def rmse(y_true, y_pred):
    """Custom RMSE regression metric."""
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))


def build_model(hparams):
    """Build and compile a TensorFlow Keras DNN Regressor."""

    feature_columns = [
        tf.feature_column.numeric_column(key=feature)
        for feature in NUMERIC_FEATURES
    ]
    
    input_layers = {
        feature.key: tf.keras.layers.Input(name=feature.key, shape=(), dtype=tf.float32)
        for feature in feature_columns
    }
    # Keras Functional API: https://keras.io/guides/functional_api
    inputs = tf.keras.layers.DenseFeatures(feature_columns, name='inputs')(input_layers)
    d1 = tf.keras.layers.Dense(256, activation=tf.nn.relu, name='d1')(inputs)
    d2 = tf.keras.layers.Dropout(hparams['dropout'], name='d2')(d1)    
    #Nota: uma saída escalar de um único neurônio para regressão.
    output = tf.keras.layers.Dense(1, name='output')(d2)
    
    model = tf.keras.Model(input_layers, output, name='online-retail-clv')
    
    optimizer = tf.keras.optimizers.Adam(hparams['learning-rate'])    
    
    # Observação: a perda de MAE é mais resistente a valores discrepantes do que MSE.
    model.compile(loss=tf.keras.losses.MAE,
                  optimizer=optimizer,
                  metrics=[['mae', 'mse', rmse]])
    
    return model


def train_evaluate_explain_model(hparams):
    """Train, evaluate, explain TensorFlow Keras DNN Regressor.
    Args:
      hparams(dict): A dictionary containing model training arguments.
    Returns:
      history(tf.keras.callbacks.History): Keras callback that records training event history.
    """
    training_ds = read_bigquery(*caip_uri_to_fields(hparams['training-data-uri'])).prefetch(1).shuffle(hparams['batch-size']*10).batch(hparams['batch-size']).repeat()
    eval_ds = read_bigquery(*caip_uri_to_fields(hparams['validation-data-uri'])).prefetch(1).shuffle(hparams['batch-size']*10).batch(hparams['batch-size'])
    test_ds = read_bigquery(*caip_uri_to_fields(hparams['test-data-uri'])).prefetch(1).shuffle(hparams['batch-size']*10).batch(hparams['batch-size'])
    
    model = build_model(hparams)
    logging.info(model.summary())
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=hparams['tensorboard-dir'],
        histogram_freq=1)
    
    # Reduza o overfitting e diminua os tempos de treinamento.
    earlystopping_callback = tf.keras.callbacks.EarlyStopping(patience=2)
    
    #Garanta a resiliência do seu trabalho de treinamento para reinicializações de VM.
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath= hparams['checkpoint-dir'],
        save_weights_only=True,
        monitor='val_loss',
        mode='min')
    
    # Padrão de design de épocas virtuais:
    # https://medium.com/google-cloud/ml-design-pattern-3-virtual-epochs-f842296de730
    TOTAL_TRAIN_EXAMPLES = int(hparams['stop-point'] * hparams['n-train-examples'])
    STEPS_PER_EPOCH = (TOTAL_TRAIN_EXAMPLES // (hparams['batch-size']*hparams['n-checkpoints']))    
    
    history = model.fit(training_ds,
                        validation_data=eval_ds,
                        steps_per_epoch=STEPS_PER_EPOCH,
                        epochs=hparams['n-checkpoints'],
                        callbacks=[[tensorboard_callback,
                                    earlystopping_callback,
                                    checkpoint_callback]])
    
    logging.info(model.evaluate(test_ds))
    
    # Crie um diretório temporário para salvar o TF SavedModel intermediário antes da criação de metadados Explainable.
    tmpdir = tempfile.mkdtemp()
    
    # Exporte o modelo Keras no formato TensorFlow SavedModel.
    model.save(tmpdir)
    
    #Anote e salve TensorFlow SavedModel com metadados Explainable no GCS.
    builder = SavedModelMetadataBuilder(tmpdir)
    builder.save_model_with_metadata(hparams['model-dir'])
    
    return history

#### 2. Escreva um arquivo `task.py` como um ponto de entrada para seu contêiner de modelo de ML personalizado

In [ ]:
%%writefile {MODEL_NAME}/trainer/task.py
import os
import argparse

from trainer import model

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    # Argumentos de treinamento de contêiner personalizado da Vertex. Eles são definidos pela Vertex AI durante o treinamento, mas também podem ser substituídos.
    parser.add_argument('--model-dir', dest='model-dir',
                        default=os.environ['AIP_MODEL_DIR'], type=str, help='Model dir.')
    parser.add_argument('--checkpoint-dir', dest='checkpoint-dir',
                        default=os.environ['AIP_CHECKPOINT_DIR'], type=str, help='Checkpoint dir set during Vertex AI training.')    
    parser.add_argument('--tensorboard-dir', dest='tensorboard-dir',
                        default=os.environ['AIP_TENSORBOARD_LOG_DIR'], type=str, help='Tensorboard dir set during Vertex AI training.')    
    parser.add_argument('--data-format', dest='data-format',
                        default=os.environ['AIP_DATA_FORMAT'], type=str, help="Tabular data format set during Vertex AI training. E.g.'csv', 'bigquery'")
    parser.add_argument('--training-data-uri', dest='training-data-uri',
                        default=os.environ['AIP_TRAINING_DATA_URI'], type=str, help='Training data GCS or BQ URI set during Vertex AI training.')
    parser.add_argument('--validation-data-uri', dest='validation-data-uri',
                        default=os.environ['AIP_VALIDATION_DATA_URI'], type=str, help='Validation data GCS or BQ URI set during Vertex AI training.')
    parser.add_argument('--test-data-uri', dest='test-data-uri',
                        default=os.environ['AIP_TEST_DATA_URI'], type=str, help='Test data GCS or BQ URI set during Vertex AI training.')
    # Argumentos de treinamento do modelo.
    parser.add_argument('--learning-rate', dest='learning-rate', default=0.001, type=float, help='Learning rate for optimizer.')
    parser.add_argument('--dropout', dest='dropout', default=0.2, type=float, help='Float percentage of DNN nodes [0,1] to drop for regularization.')    
    parser.add_argument('--batch-size', dest='batch-size', default=16, type=int, help='Number of examples during each training iteration.')    
    parser.add_argument('--n-train-examples', dest='n-train-examples', default=2638, type=int, help='Number of examples to train on.')
    parser.add_argument('--stop-point', dest='stop-point', default=10, type=int, help='Number of passes through the dataset during training to achieve convergence.')
    parser.add_argument('--n-checkpoints', dest='n-checkpoints', default=10, type=int, help='Number of model checkpoints to save during training.')
    
    args = parser.parse_args()
    hparams = args.__dict__

    model.train_evaluate_explain_model(hparams)

#### 3. Escreva um `Dockerfile` para seu contêiner de modelo de ML personalizado

Em terceiro lugar, você escreverá um `Dockerfile` que contém o código do modelo, bem como especifica as dependências do código do modelo.

Observe que a imagem base abaixo é um [contêiner Google Cloud Deep Learning](https://cloud.google.com/vertex-ai/docs/general/deep-learning) que contém versões testadas e otimizadas de dependências de código de modelo, como ` tensorflow` e o SDK `google-cloud-bigquery`.

In [ ]:
%%writefile {MODEL_NAME}/Dockerfile
# Specifies base image and tag.
# https://cloud.google.com/vertex-ai/docs/general/deep-learning
# https://cloud.google.com/deep-learning-containers/docs/choosing-container
FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-3

# Define o diretório de trabalho do contêiner.
WORKDIR /root

# Copia o requirements.txt no contêiner para reduzir as chamadas de rede.
COPY requirements.txt .
# Installs additional packages.
RUN pip3 install -U -r requirements.txt

# Copia o código do treinador para a imagem do docker.
COPY . /trainer

# Define o diretório de trabalho do contêiner.
WORKDIR /trainer

# Define o diretório de trabalho do contêiner.
ENTRYPOINT ["python", "-m", "trainer.task"]

### 4. Escreva um arquivo `requirements.txt` para especificar dependências de código ML adicionais

Essas são dependências adicionais para o código do seu modelo fora dos contêineres de aprendizado profundo necessários para a explicabilidade da previsão e o leitor BigQuery TensorFlow IO.

In [ ]:
%%writefile {MODEL_NAME}/requirements.txt
explainable-ai-sdk==1.3.0
tensorflow-io==0.15.0
pyarrow

#### 5. Use o Cloud Build para criar e enviar seu contêiner para o Google Cloud Artifact Registry

Em seguida, você usará o [Cloud Build](https://cloud.google.com/build) para criar e enviar seu contêiner de modelo personalizado do TensorFlow para o [Google Cloud Artifact Registry](https://cloud.google.com/artifact-registro).

O Cloud Build traz reutilização e automação para sua experimentação de ML, permitindo que você crie, teste e implante de maneira confiável seu código de modelo de ML como parte de um fluxo de trabalho de CI/CD. O Artifact Registry fornece um repositório centralizado para você armazenar, gerenciar e proteger suas imagens de contêiner de ML. Isso permitirá que você compartilhe com segurança seu trabalho de ML com outras pessoas e reproduza os resultados do experimento.

**Observação**: a etapa inicial de compilação e envio levará cerca de 20 minutos, mas o Cloud Build pode aproveitar o armazenamento em cache para compilações subsequentes.

#### Criar repositório de artefatos para imagens de contêiner personalizadas

In [ ]:
ARTIFACT_REPOSITORY="online-retail-clv"

In [ ]:
# Create an Artifact Repository using the gcloud CLI.
!gcloud artifacts repositories create $ARTIFACT_REPOSITORY \
--repository-format=docker \
--location=$REGION \
--description="Artifact registry for ML custom training images for predictive CLV"

#### Criar instruções `cloudbuild.yaml`

In [ ]:
IMAGE_NAME="dnn-regressor"
IMAGE_TAG="latest"
IMAGE_URI=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{ARTIFACT_REPOSITORY}/{IMAGE_NAME}:{IMAGE_TAG}"

In [ ]:
cloudbuild_yaml = f"""steps:
- name: 'gcr.io/cloud-builders/docker'
  args: [ 'build', '-t', '{IMAGE_URI}', '.' ]
images: 
- '{IMAGE_URI}'"""

with open(f"{MODEL_NAME}/cloudbuild.yaml", "w") as fp:
    fp.write(cloudbuild_yaml)

#### Crie e envie sua imagem de contêiner para seu repositório de artefatos

In [ ]:
!gcloud builds submit --timeout=20m --config {MODEL_NAME}/cloudbuild.yaml {MODEL_NAME}

Agora que seu contêiner personalizado foi criado e armazenado em seu Artifact Registry, é hora de treinar nosso modelo na nuvem com a Vertex AI.

## Execute um trabalho de treinamento personalizado na Vertex AI

### 1. Crie uma instância Vertex Tensorboard para rastrear seus experimentos de modelo

[**Vertex TensorBoard**](https://cloud.google.com/vertex-ai/docs/experiments) é a versão gerenciada do Google Cloud do [**TensorBoard**](https://www.tensorflow.org/tensorboard) para visualização experimental de ML. Com o Vertex TensorBoard, você pode acompanhar, visualizar e comparar experimentos de ML e compartilhá-los com sua equipe. Além das visualizações poderosas do TensorBoard de código aberto, o Vertex TensorBoard oferece:

* Um link persistente e compartilhável para o painel do seu experimento.
* Uma lista pesquisável de todos os experimentos em um projeto.
* Integrações com serviços Vertex AI para avaliação de treinamento de modelo.

In [ ]:
!gcloud beta ai tensorboards create \
--display-name=$MODEL_NAME --region=$REGION

In [ ]:
TENSORBOARD_RESOURCE_NAME= !(gcloud beta ai tensorboards list --region=$REGION --format="value(name)")
TENSORBOARD_RESOURCE_NAME= TENSORBOARD_RESOURCE_NAME[1]
TENSORBOARD_RESOURCE_NAME

### 2. Execute seu trabalho de treinamento de contêiner personalizado

Use a classe `CustomTrainingJob` para definir o trabalho, que usa os seguintes parâmetros específicos para treinamento de contêiner personalizado:

* `display_name`: Seu nome definido pelo usuário deste pipeline de treinamento.
* `container_uri`: O URI de sua imagem de contêiner de treinamento personalizado.
* `model_serving_container_image_uri`: O URI de um contêiner que pode servir previsões para seu modelo. Você usará um contêiner pré-criado Vertex.

Use a função `run()` para iniciar o treinamento, que leva os seguintes parâmetros:

* `replica_count`: O número de réplicas do worker.
* `model_display_name`: O nome de exibição do modelo se o script produzir um modelo gerenciado.
* `machine_type`: O tipo de máquina a ser usada para treinamento.
* `bigquery_destination`: o URI do BigQuery em que o conjunto de dados Tabular criado é gravado.
* `predefined_split_column_name`: como este laboratório utilizou o BigQuery para processamento e divisão de dados, esta coluna é especificada para indicar divisões de dados.

A função run cria um pipeline de treinamento que treina e cria um objeto Vertex `Model`. Após a conclusão do pipeline de treinamento, a função `run()` retorna o objeto `Model`.

Nota: este `CustomContainerTrainingJob` levará cerca de 20 minutos para provisionar recursos e treinar seu modelo.

In [ ]:
# args de linha de comando para trainer.task definido acima. Revise o argumento 'ajuda' para obter uma descrição.
# Você definirá os argumentos de treinamento do modelo abaixo. A Vertex AI definirá as variáveis de ambiente para URIs de treinamento.
CMD_ARGS= [
    "--learning-rate=" + str(0.001),
    "--batch-size=" + str(16),
    "--n-train-examples=" + str(2638),
    "--stop-point=" + str(10),
    "--n-checkpoints=" + str(10),
    "--dropout=" + str(0.2),   
]

In [ ]:
# Ao definir BASE_OUTPUT_DIR, o Vertex AI definirá as variáveis de ambiente AIP_MODEL_DIR, AIP_CHECKPOINT_DIR, AIP_TENSORBOARD_LOG_DIR
# durante o treinamento para o seu código de treinamento de ML gravar.
TIMESTAMP=datetime.datetime.now().strftime('%Y%m%d%H%M%S')
BASE_OUTPUT_DIR= f"gs://{GCS_BUCKET}/vertex-custom-training-{MODEL_NAME}-{TIMESTAMP}"

In [ ]:
job = aiplatform.CustomContainerTrainingJob(
    display_name="online-retail-clv-3M-dnn-regressor",
    container_uri=IMAGE_URI,
    # https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
    # gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest
    model_serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest",
)

model = job.run(
    dataset=tabular_dataset,
    model_display_name=MODEL_NAME,
    # Diretório de saída de trabalho personalizado do GCS.
    base_output_dir=BASE_OUTPUT_DIR,
    # as divisões do conjunto de dados BQ Tabular serão gravadas em seu próprio conjunto de dados BQ para reprodutibilidade.
    bigquery_destination=f"bq://{PROJECT_ID}",
    # isso corresponde à coluna de divisão de dados do BigQuery.
    predefined_split_column_name="data_split",
    # os argumentos de linha de comando de treinamento do modelo definidos em trainer.task.
    args=CMD_ARGS,
    # Argumentos de WorkerPool de tarefa personalizada.
    replica_count=1,
    machine_type="c2-standard-4",
    # Forneça o nome do recurso do Tensorboard para gravar logs do Tensorboard durante o treinamento.
    tensorboard=TENSORBOARD_RESOURCE_NAME,
    # Forneça sua conta de serviço de treinamento personalizada da Vertex criada durante a configuração do laboratório.
    service_account=f"vertex-custom-training-sa@{PROJECT_ID}.iam.gserviceaccount.com"
)

### 3. Inspecione o desempenho do treinamento do modelo com o Vertex TensorBoard

Você pode visualizar os logs do seu modelo na Vertex AI [**guia Experimentos**](https://console.cloud.google.com/vertex-ai/experiments) no Console do Cloud. Clique no link **Abrir Tensorboard**. Você será solicitado a autenticar com sua conta do Google do Qwiklabs antes que uma página do Vertex Tensorboard seja aberta em uma guia do navegador. Assim que seu modelo começar a treinar, você verá suas métricas de avaliação de treinamento gravadas neste painel que você pode inspecionar durante a execução do treinamento, bem como após a conclusão do trabalho.

Observação: o Tensorboard fornece uma ferramenta de depuração valiosa para inspecionar o desempenho do seu modelo durante e após o treinamento do modelo. O modelo deste laboratório treina em menos de um minuto e, às vezes, é concluído antes que os logs terminem de aparecer no Tensorboard. Se for esse o caso, atualize a janela quando o trabalho de treinamento for concluído para ver a avaliação de desempenho do seu modelo.

## Sirva seu modelo com o Vertex AI Prediction: previsões e explicações de modelos on-line

Você tem um modelo treinado no GCS agora, vamos fazer a transição para servir nosso modelo com o Vertex AI Prediction para previsões e explicações de modelos on-line.

### 1. Crie os metadados e parâmetros de explicação

[**Vertex Explainable AI**](https://cloud.google.com/vertex-ai/docs/explainable-ai) integra atribuições de recursos à Vertex AI. Vertex Explainable AI ajuda você a entender as saídas do seu modelo para tarefas de classificação e regressão. A Vertex AI informa quanto cada recurso nos dados contribuiu para o resultado previsto. Você pode então usar essas informações para verificar se o modelo está se comportando conforme o esperado, identificar e mitigar vieses em seus modelos e obter ideias de maneiras de melhorar seu modelo e seus dados de treinamento.

Você recuperará essas atribuições de recurso para obter informações sobre as previsões de CLV do seu modelo.

In [ ]:
DEPLOYED_MODEL_DIR = os.path.join(BASE_OUTPUT_DIR, 'model')

In [ ]:
loaded = tf.keras.models.load_model(DEPLOYED_MODEL_DIR)

In [ ]:
serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys())[0]

serving_output = list(loaded.signatures["serving_default"].structured_outputs.keys())[0]

feature_names = [
    "n_purchases",
    "avg_purchase_size",
    "avg_purchase_revenue",
    "customer_age",
    "days_since_last_purchase"
]

In [ ]:
# Especifique o método de atribuição do recurso Shapley amostrado com o parâmetro path_count
# controlando o número de permutações de recursos a serem consideradas ao aproximar os valores de Shapley.

explain_params = aiplatform.explain.ExplanationParameters(
    {"sampled_shapley_attribution": {"path_count": 10}}
)

In [ ]:
# https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/ExplanationSpec
input_metadata = {
    "input_tensor_name": serving_input,
    "encoding": "BAG_OF_FEATURES",
    "modality": "numeric",
    "index_feature_mapping": feature_names,
}

output_metadata = {"output_tensor_name": serving_output}

input_metadata = aiplatform.explain.ExplanationMetadata.InputMetadata(input_metadata)
output_metadata = aiplatform.explain.ExplanationMetadata.OutputMetadata(output_metadata)

explain_metadata = aiplatform.explain.ExplanationMetadata(
    inputs={"features": input_metadata}, outputs={"medv": output_metadata}
)

## Implante um Vertex `Endpoint` para previsões online

Antes de usar seu modelo para fazer previsões, você precisa implantá-lo em um objeto `Endpoint`. Ao implantar um modelo em um `Endpoint`, você associa recursos físicos (máquina) a esse modelo para permitir que ele forneça previsões online. As previsões online têm requisitos de baixa latência; fornecer recursos para o modelo com antecedência reduz a latência. Você pode fazer isso chamando a função deploy no recurso `Model`. Isso fará duas coisas:

1. Crie um recurso `Endpoint` para implantar o recurso `Model`.
2. Implante o recurso `Model` no recurso `Endpoint`.

A função `deploy()` recebe os seguintes parâmetros:

* `deployed_model_display_name`: Um nome legível por humanos para o modelo implantado.
* `traffic_split`: Porcentagem de tráfego no endpoint que vai para este modelo, que é especificado como um dicionário de um ou mais pares chave/valor. Se for apenas um modelo, especifique como { "0": 100 }, onde "0" refere-se a este modelo sendo carregado e 100 significa 100% do tráfego.
* `machine_type`: O tipo de máquina a ser usada para treinamento.
* `accelerator_type`: O tipo de acelerador de hardware.
* `accelerator_count`: O número de aceleradores a serem anexados a uma réplica de trabalho.
* `starting_replica_count`: O número de instâncias de computação para provisionamento inicial.
* `max_replica_count`: O número máximo de instâncias de computação para escalar. Neste laboratório, apenas uma instância é provisionada.
* `explanation_parameters`: Metadados para configurar o método de aprendizado Explainable AI.
* `explanation_metadata`: Metadados que descrevem seu modelo do TensorFlow para o Explainable AI, como recursos, tensores de entrada e saída.

Observação: isso pode levar cerca de 5 minutos para provisionar recursos de previsão para seu modelo.

In [ ]:
endpoint = model.deploy(
    traffic_split={"0": 100},
    machine_type="n1-standard-2",
    explanation_parameters=explain_params,
    explanation_metadata=explain_metadata
)

## Get an online prediction and explanation from deployed model

Por fim, você usará seu `Endpoint` para recuperar previsões e atribuições de recursos. Esta é uma instância do cliente recuperada do conjunto de teste.

In [ ]:
# actual: 3181.04
test_instance_dict = {
    "n_purchases": 2,
    "avg_purchase_size": 536.5,
    "avg_purchase_revenue": 1132.7,
    "customer_age": 123,
    "days_since_last_purchase": 32,
}

Para solicitar previsões, você chama o método `predict()`.

In [ ]:
endpoint.predict([test_instance_dict])

Para recuperar explicações (previsões + atribuições de recursos), chame o método `explain()`.

In [ ]:
explanations = endpoint.explain([test_instance_dict])

In [ ]:
pd.DataFrame.from_dict(explanations.explanations[0].attributions[0].feature_attributions, orient='index').plot(kind='barh');

Com base nas atribuições de recursos para essa previsão, seu modelo descobriu que a receita média de compra e a idade do cliente tiveram a maior contribuição marginal na previsão do valor monetário desse cliente durante o período de teste de três meses. Ele também identificou os dias relativamente longos desde a última compra como um impacto negativo na previsão. Usando esses insights, você pode planejar um experimento para avaliar as intervenções de marketing direcionadas para esse cliente recorrente, como descontos por volume, para incentivar esse cliente a comprar com mais frequência a fim de gerar receita adicional.

## Próximos passos

Parabéns! Neste laboratório, você percorreu um fluxo de trabalho de experimentação de aprendizado de máquina usando o BigQuery do Google Cloud para armazenamento e análise de dados e os serviços de aprendizado de máquina Vertex AI para treinar e implantar um modelo do TensorFlow para prever o valor da vida útil do cliente. Você evoluiu do treinamento de um modelo do TensorFlow localmente para o treinamento na nuvem com a Vertex AI e aproveitou vários novos recursos de plataforma unificada, como atribuições de recursos de previsão Vertex TensorBoard e Explainable AI.

## License

In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.